# Modèle 1 : Architectures et Models comparaison

## Définition d'une procédure de test

**Dataset :**

Afin de pouvoir comparer nos architectures et par la suite nos modèles, nous avons généré un dataset d'entrainement.
Nous avons utilisé Chat-GPT en lui donnant des Id_solution ainsi que toutes les informations que nous utilisons dans notre modèle (Titre, Description, Bilan énergie) afin qu'il génère des requêtes qu'un utilisateur aurrait pu formuler en fonction de ces solutions. Nous avons tiré les id_solution de manière aléatoire afin d'avoir une répartition des question homogène sur le dataset. Nous n'avons pas de doublons d'id_solution dans le dataset.

**Méthode de test :**

Afin de tester l'accuracy de nos modèles, nous avons choisi de faire des inférence sur notre dataset de test et de compter 1 point si l'id_solution réel est prédit en première position et 0 point sinon. Ensuite on divise par le nombre de données dans le dataset afin d'avoir une valeur d'accuracy entre 0 et 1.

Il est evident que notre modèle prédira en 2, 3, ... positions d'autres résultats pertinents que l'utilisateur pourrait étudier, mais cela n'étant pas évident à quantifier dans un premier test automatique, nous avons choisi de ne pas le traiter pour le moment.

## Étude préliminaire

Dans un premier temps nous avons tester différents type de model sur la première architecture afin de déterminer si un modèle multi-lingue était aussi bon en Français qu'un modèle purement Français. Durant cette étude préliminaire, nous n'avions pas crée de dataset de test et de fonction d'accuracy, nous avons donc comparé les résultats à l'oeil sur le dataset fourni par Kerdos. Il en ressort que les meilleurs modèles sont les plus gros, avec *paraphrase-multilingual-mpnet-base-v2*, *sentence-transformers/LaBSE* et *dangvantuan/sentence-camembert-large*. Notre modèle Français légèrement meilleur mais ne permet pas l'implémentation du multi-lingue. Nous avons donc choisi de nous concentrer sur les deux autres modèles dans la suite de cette étude.

| Nom modèle | Langage | Temps génération embedding | Taille embedding | Temps inférence (1 reqûete) |
|------------|---------|----------------------------|------------------|----------------|
| paraphrase-multilingual-MiniLM-L12-v2 | Multi | 2min| 1.7MB| 0.1s|
| paraphrase-multilingual-mpnet-base-v2 | Multi | 8min30| 3.4MB| 0.1s|
| sentence-transformers/LaBSE | Multi | 10min| 3.4MB| 0.1s|
| dangvantuan/sentence-camembert-base | Fr | 6min|3.4MB| 0.1s|
| dangvantuan/sentence-camembert-large| Fr | 14min|4.5MB| 0.1s|

Le dataset des solutions étudiés à légèrement évolué entre cette étude préliminaire et les tests suivants d'oû quelques différences.

## Architectures

Nous avons fais le choix de tester 3 architectures différentes :

### Architecture 1
- **Sens sémentique de la requête de l'utilisateur :** Nous calculons un vecteur pour chaque phrases et nous faisons la moyenne. L'idée était d'avoir le sens sémentique globale de la requête.

- **Sens sémentique des solutions :** Nous calculons un vecteur globale pour la solution en faisant la moyenne des vecteurs de toutes ses phrases afin d'avoir le sens globale de la solution.

### Architecture 2
- **Sens sémentique de la requête de l'utilisateur :** *idem*

- **Sens sémentique des solutions :** Nous comparons chaque phrase indépendament avec l'entrée de l'utilisateur pour ne garder que le max de la correspondance sémentique. Pour cette architecture on est parti du principe que lorsqu'on cherche des informations dans un document, on peut être intéressé par une partie en particulier.

### Architecture 3
- **Sens sémentique de la requête de l'utilisateur :** *idem*

- **Sens sémentique des solutions :** Nous comparons chaque section (titre, description, etc), indépendament avec l'entrée de l'utilisateur pour ne garder que le max de la correspondance sémentique.


## Comparatif 

| Modèle | Architecture | Temps génération embedding | Taille embedding | Temps inférence (50 requêtes) | Accuracy |
|--------|------------|----------------------------|------------------|----------------|-------------|
|paraphrase-multilingual-mpnet-base-v2| Architecture 1 | 7min | 3.4MB | 2.5s | 0.56 |
|paraphrase-multilingual-mpnet-base-v2| Architecture 2 | 7min | 33.2 MB | 10s| 0.72 |
|paraphrase-multilingual-mpnet-base-v2| Architecture 3 | 5min| 11.4 MB| 8s | 0.88 |
|sentence-transformers/LaBSE| Architecture 1 | 6min| 3.4 MB| 3s | 0.64 |
|sentence-transformers/LaBSE| Architecture 2 | 6min15 | 33.2 MB | 10s | 0.74 |
|sentence-transformers/LaBSE| Architecture 3 | 5min | 11.4 MB | 8s | 0.82 |

Il en ressort que le meilleur modèle choix est t'utiliser le modèle **paraphrase-multilingual-mpnet-base-v2** avec **l'architecture 3**. Ce modèle donne la meilleure accuracy, de plus l'architecture 3 est un bon compromis en taille d'embedding avec seulement 11.4MB.

## Ajout du secteur d'activité dans la requête utilisateur

Dans l'étude ci-dessus nous n'avions pas pris en compte le secteur d'activité que donne l'utilisateur pour prédire nos solutions, ici nous étutions l'impacte de ce paramètre :

| Inclure secteur d'activité | Modèle | Architecture | Temps inférence (50 requêtes) | Accuracy |
|----------------------------|--------|------------|---------------------------------|----------|
| Oui |paraphrase-multilingual-mpnet-base-v2| Architecture 3 | 10s| 0.86 |
| Non |paraphrase-multilingual-mpnet-base-v2| Architecture 3 | 8s | 0.88 |

On en conclu donc qu'il n'y a pas d'inétrêt majeur à intégrer le secteur d'activité à nos requêtes. Cependant il est important de prendre en compte la piètre qualité de notre dataset pour réaliser ce test. En effet, ChatGPT4 s'est montré très performant pour générer des requête potentielles en fonction des informations des solutions, mais bien moins performant pour déduire des secteurs d'activité.

## Autres axes étudiés mais pas implémentés

### Fine-tunning

### Quantization de l'embedding